In [11]:
import torch
import torch.nn as nn
import numpy as np
import os
import math
import warnings
import json
import cv2
import mediapipe as mp

# --- PHẦN 1: ĐỊNH NGHĨA LẠI KIẾN TRÚC MÔ HÌNH ---
# (Không thay đổi - Giống hệt code trước)
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class KeypointTransformerClassifier(nn.Module):
    def __init__(self, num_classes, d_model, nhead, num_encoder_layers,
                 dim_feedforward, dropout, input_features=1662, max_seq_len=128):
        super(KeypointTransformerClassifier, self).__init__()
        self.d_model = d_model
        self.input_embedding = nn.Linear(input_features, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len=max_seq_len)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward,
            dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, num_layers=num_encoder_layers
        )
        self.classifier_head = nn.Linear(d_model, num_classes)

    def forward(self, src):
        src = self.input_embedding(src) * math.sqrt(self.d_model)
        src = src.permute(1, 0, 2)
        src = self.pos_encoder(src)
        src = src.permute(1, 0, 2)
        output = self.transformer_encoder(src)
        output = output.mean(dim=1)
        output = self.classifier_head(output)
        return output

# --- PHẦN 2: CÁC HÀM HỖ TRỢ XỬ LÝ VIDEO & KEYPOINT ---
# (Không thay đổi - Giống hệt code trước)
mp_holistic = mp.solutions.holistic
NUM_POSE_LANDMARKS = 33
NUM_FACE_LANDMARKS = 468
NUM_HAND_LANDMARKS = 21
ZERO_POSE = np.zeros(NUM_POSE_LANDMARKS * 4)
ZERO_FACE = np.zeros(NUM_FACE_LANDMARKS * 3)
ZERO_HAND = np.zeros(NUM_HAND_LANDMARKS * 3)

def extract_holistic_keypoints(video_frames_rgb_list):
    all_frame_keypoints = []
    with mp_holistic.Holistic(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:

        for frame in video_frames_rgb_list:
            results = holistic.process(frame)
            pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else ZERO_POSE
            face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else ZERO_FACE
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else ZERO_HAND
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else ZERO_HAND
            frame_keypoints = np.concatenate([pose, face, lh, rh])
            all_frame_keypoints.append(frame_keypoints)

    return np.array(all_frame_keypoints).astype(np.float32)

def read_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Lỗi: Không thể mở file video: {video_path}")
        return None

    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        all_frames.append(frame_rgb)
    cap.release()
    return all_frames

# --- PHẦN 3: HÀM SUY LUẬN (INFERENCE) CHÍNH ---
# *** ĐÃ SỬA ĐỔI LOGIC TRONG HÀM NÀY ***

def predict_video(model, video_path, label_map_inverse, device, max_seq_len, num_features):
    """
    Pipeline hoàn chỉnh: Đọc video, trích xuất, tiền xử lý (CENTER TRIM), và dự đoán.
    """
    model.eval()

    # 1. Đọc video -> list các frame
    frames = read_video_frames(video_path)
    if frames is None or len(frames) == 0:
        return "Lỗi: Không thể đọc video", 0.0

    # 2. Trích xuất Keypoints (List[frame] -> mảng NumPy (T, 1662))
    keypoints = extract_holistic_keypoints(frames)

    # 3. Tiền xử lý (Đệm/Cắt)
    num_frames = keypoints.shape[0]

    # =========================================================
    # --- BẮT ĐẦU THAY ĐỔI LOGIC (Center Trim) ---
    if num_frames == max_seq_len:
        # Vừa đủ, không làm gì
        pass
    elif num_frames > max_seq_len:
        # TRƯỚC ĐÂY (Cắt đầu):
        # keypoints = keypoints[:max_seq_len, :]

        # HIỆN TẠI (Cắt giữa):
        # 1. Tính toán điểm bắt đầu để lấy clip ở giữa
        start_idx = (num_frames - max_seq_len) // 2

        # 2. Lấy lát cắt từ giữa
        keypoints = keypoints[start_idx : start_idx + max_seq_len, :]

    else: # num_frames < max_seq_len
        # Đệm (Pad) nếu ngắn hơn (Logic này không đổi)
        padding_needed = max_seq_len - num_frames
        padding_tensor = np.zeros((padding_needed, num_features), dtype=np.float32)
        keypoints = np.concatenate([keypoints, padding_tensor], axis=0)
    # --- KẾT THÚC THAY ĐỔI LOGIC ---
    # =========================================================

    # 4. Chuyển sang Tensor và thêm chiều Batch (B=1)
    input_tensor = torch.from_numpy(keypoints)
    input_tensor = input_tensor.unsqueeze(0).to(device)

    # 5. Suy luận
    with torch.no_grad():
        outputs = model(input_tensor)

        # 6. Hậu xử lý: Lấy xác suất
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted_id = torch.max(probabilities, 1)

        predicted_id = predicted_id.item()
        confidence = confidence.item()

        # 7. Map ID sang Tên nhãn (ví dụ: 5 -> 'hello')
        prediction_name = label_map_inverse.get(predicted_id, "KHÔNG RÕ NHÃN")

        return prediction_name, confidence

# --- PHẦN 4: THỰC THI SCRIPT ---
# (Không thay đổi - Giống hệt code trước)

if __name__ == "__main__":
    warnings.filterwarnings("ignore", category=UserWarning)

    # --- CÁC THAM SỐ (PHẢI GIỐNG HỆT FILE HUẤN LUYỆN) ---

    # *** (QUAN TRỌNG) HÃY SỬA 3 ĐƯỜNG DẪN NÀY ***
    LABEL_MAP_PATH = "label_map.json"
    MODEL_PATH = "best_transformer_model.pth"
    VIDEO_TO_TEST = "Video_Internet/father.mp4"

    # --------------------------------------------------

    # Tham số mô hình (PHẢI GIỐNG HỆT FILE HUẤN LUYỆN)
    NUM_FEATURES = 1662
    MAX_SEQ_LEN = 128
    D_MODEL = 512
    NHEAD = 8
    NUM_ENCODER_LAYERS = 4
    DIM_FEEDFORWARD = 2048
    DROPOUT = 0.1

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Sử dụng thiết bị: {DEVICE}")

    # --- Bước 1: Tải và Tái tạo Label Map ---
    label_map = None
    label_map_inverse = None
    NUM_CLASSES = 0

    print(f"Đang tải bản đồ nhãn từ: '{LABEL_MAP_PATH}'...")
    try:
        with open(LABEL_MAP_PATH, 'r', encoding='utf-8') as f:
            label_map = json.load(f)

        NUM_CLASSES = len(label_map)
        label_map_inverse = {idx: label for label, idx in label_map.items()}
        print(f"Tải và tái tạo {NUM_CLASSES} nhãn thành công.")

    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file bản đồ nhãn: '{LABEL_MAP_PATH}'")
    except Exception as e:
        print(f"Lỗi khi đọc file JSON: {e}")

    # Chỉ tiếp tục nếu tải bản đồ nhãn thành công
    if label_map and label_map_inverse:

        # --- Bước 2: Tải Mô hình ---
        print(f"Đang tải mô hình từ '{MODEL_PATH}'...")
        try:
            model = KeypointTransformerClassifier(
                num_classes=NUM_CLASSES, d_model=D_MODEL, nhead=NHEAD,
                num_encoder_layers=NUM_ENCODER_LAYERS, dim_feedforward=DIM_FEEDFORWARD,
                dropout=DROPOUT, input_features=NUM_FEATURES, max_seq_len=MAX_SEQ_LEN
            )
            model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
            model.to(DEVICE)
            model.eval()
            print("Tải mô hình thành công.")

            # --- Bước 3: Chạy Suy luận ---
            print(f"\n--- Bắt đầu dự đoán video: {VIDEO_TO_TEST} ---")
            if not os.path.exists(VIDEO_TO_TEST):
                print(f"LỖI: Không tìm thấy file video: {VIDEO_TO_TEST}")
            else:
                prediction, confidence = predict_video(
                    model=model,
                    video_path=VIDEO_TO_TEST,
                    label_map_inverse=label_map_inverse,
                    device=DEVICE,
                    max_seq_len=MAX_SEQ_LEN,
                    num_features=NUM_FEATURES
                )

                print("\n================ KẾT QUẢ DỰ ĐOÁN ================")
                print(f"== Video: {VIDEO_TO_TEST}")
                print(f"== Dự đoán: {prediction.upper()}")
                print(f"== Độ tự tin: {confidence * 100:.2f}%")
                print("==================================================")

        except FileNotFoundError:
            print(f"LỖI: Không tìm thấy file trọng số '{MODEL_PATH}'")
        except RuntimeError as e:
            print(f"LỖI khi tải mô hình: {e}")
        except Exception as e:
            print(f"Đã xảy ra lỗi không xác định: {e}")

Sử dụng thiết bị: cpu
Đang tải bản đồ nhãn từ: 'label_map.json'...
Tải và tái tạo 47 nhãn thành công.
Đang tải mô hình từ 'best_transformer_model.pth'...
Tải mô hình thành công.

--- Bắt đầu dự đoán video: Video_Internet/father.mp4 ---

================ KẾT QUẢ DỰ ĐOÁN ================
== Video: Video_Internet/father.mp4
== Dự đoán: NO
== Độ tự tin: 97.87%
